In [3]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import time

In [4]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta = "C:/Users/cristian.guatemal/Documents/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [5]:
#Importación de archivos de afiliados
inicio = time.time()
afi = pd.read_csv( ruta_afi, delimiter='\t', encoding='iso-8859-1', decimal='.')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  1.0  minutos y 46.28170609474182  segundos


In [15]:
#Agrupamiento por relación de trabajo
inicio = time.time()
conditions = [ (afi['SECTOR'].isin(['PRIVADO', 'PASANTE PRIVADO'])), 
               (afi['SECTOR'].isin(['PUBLICO', 'PASANTE PUBLICO'])), 
               (afi['SECTOR'].isin(['INDEPENDIENTES', '108-ARTISTA Y GESTOR DE CULTURA', 
                                     '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  '])),
               (afi['SECTOR'].isin(['69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
                                    '89-AFILIACION DOMICILIADO EN EL EXTERIOR'])),
               (afi['SECTOR'] == '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA'),
               (afi['SECTOR'] == '06-CODIGO DEL TRABAJO - CT ')
            ]
            
choices = ['PRI', 'PUB', 'IND', 'VOL_EX', 'VOL_EC', 'COD_TR']

afi['SECTOR_A'] = np.select(conditions, choices, default=afi['SECTOR'])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 17.424728393554688  segundos


In [8]:
afi['SECTOR'].unique()

array(['PRIVADO', 'PUBLICO',
       '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  ',
       'INDEPENDIENTES',
       '69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
       'PASANTE PRIVADO',
       '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA',
       '89-AFILIACION DOMICILIADO EN EL EXTERIOR', 'PASANTE PUBLICO',
       '108-ARTISTA Y GESTOR DE CULTURA', '06-CODIGO DEL TRABAJO - CT '],
      dtype=object)

In [16]:
afi['SECTOR_A'].unique()

array(['PRI', 'PUB', 'IND', 'VOL_EX', 'VOL_EC', 'COD_TR'], dtype=object)

In [ ]:
afi.shape

In [ ]:
#Para acceder a las prestaciones de IVM debe tener al menos 5 años de aporte o 60 imposiciones
ced_counts = afi['CEDULA_COD'].value_counts()

In [ ]:
# numero de cedulas que tienen menos de 60 imposiciones
ced_counts[ced_counts < 60].index.nunique()

In [ ]:
#Se filtran a las cedulas con menos de 60 imposicones 
inicio = time.time()
afi_fil = afi[~afi['CEDULA_COD'].isin(ced_counts[ced_counts < 60].index)].copy()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se crea una variable fecha
inicio = time.time()
afi_fil.loc[:, 'FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se ordenan los registros
inicio = time.time()
afi_fil = afi_fil.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, True, True] )
afi_fil.loc[:, 'FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
afi_fil.shape

In [ ]:
del afi
del ced_counts
afi_fil

In [ ]:
#Se seleccionan las características de la población para los año y mes de aportación
afi_sel = afi_fil[['CEDULA_COD', 'ANIO', 'MES', 'FECHA','SALARIO', 'SECTOR_A']].copy()
del afi_fil

In [ ]:
#Inicio de la HL
caract = afi_sel.groupby('CEDULA_COD')['FECHA'].min().reset_index() 
caract.rename( columns={'FECHA': 'INI_HL'}, inplace=True)
#Fin de la HL
caract['FIN_HL'] = caract['CEDULA_COD'].map( afi_sel.groupby('CEDULA_COD')['FECHA'].max() )
#Meses trabajados
caract['MES_T'] = caract['CEDULA_COD'].map( afi_sel['CEDULA_COD'].value_counts() )

In [ ]:
caract

In [ ]:
afi_sel =  afi_sel.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel['NUM_SEC_MES'] = 1
afi_sel['%_NUM_SECTOR'] = afi_sel['SALARIO']

In [ ]:
afi_sel

In [ ]:
#Se debe analizar a las personas que tienen varios aportes en un mismo mes
#Se suman los salarios y se concatenan los sectores para la cedula que en el mismo anio y mes tiene dos sectores
inicio = time.time()
afi_sel_g = afi_sel.groupby(['CEDULA_COD', 'ANIO', 'MES', 'SECTOR_A']).agg({'SALARIO': 'sum',
                                                                            'SECTOR_A': lambda x: ';'.join(x),
                                                                            'NUM_SEC_MES': lambda x: len(x),
                                                                            '%_NUM_SECTOR': lambda x:  ';'.join((x / x.sum()).round(4).astype(str))
                                                                           }).reset_index()
#afi_sel_g['NUM_SEC_MES'] = afi_sel.groupby(['CEDULA_COD', 'ANIO', 'MES'])['SECTOR'].nunique().values
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )


In [ ]:
afi_sel_g[afi_sel_g['NUM_SEC_MES']>1]

In [ ]:
afi_sel.groupby(['CEDULA_COD', 'ANIO', 'MES'])['SECTOR'].nunique().values

In [ ]:
#Se crea la variable que cuenta el número de sectores para cada anio y mes
inicio = time.time()
afi_sel_g['NUM_SEC_MES'] = afi_sel.groupby(['CEDULA_COD', 'ANIO', 'MES'])['SECTOR'].nunique().values
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )


In [ ]:
inicio = time.time()
#Se suma por cedula, anio, mes y sector
temp_df = afi_sel.groupby(['CEDULA_COD', 'ANIO', 'MES', 'SECTOR']).agg({'SALARIO': 'sum'}).reset_index()
sum_df = temp_df.groupby(['CEDULA_COD', 'ANIO', 'MES'])['SALARIO'].sum().reset_index()
sum_df.rename(columns={'SAL': 'TOTAL_SAL'}, inplace=True)
temp_df = temp_df.merge( sum_df, on=['CEDULA_COD', 'ANIO', 'MES'])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
sum_df = temp_df.groupby(['CEDULA_COD', 'ANIO', 'MES'])['SALARIO'].sum().reset_index()

In [ ]:
temp_df

In [ ]:
sum_df

In [ ]:
inicio = time.time()
#Se calcula el ´porcentaje del sueldo en cada sector
temp_df['PERC'] = temp_df.groupby(['CEDULA_COD', 'ANIO', 'MES'])['SALARIO'].transform(lambda x: (x / x.sum()).round(4).astype(str))

# #Se concatenan los porcentajes de sueldo de cada sector
# perc_concat = temp_df.groupby(['CEDULA_COD', 'ANIO', 'MES'])['PERC'].apply(lambda x: ';'.join(x)).reset_index()

# #Se cambia de nombre a la variable
# perc_concat.rename(columns={'PERC': '%_NUM_SECTOR'}, inplace=True)

fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
afi_sel[afi_sel['CEDULA_COD']==216].head(60)

In [ ]:
#se crean los grupos de 12 meses trabajados para tener
n_grupo = 12
afi_sel['GRUPO'] = (afi_sel.groupby('CEDULA_COD').cumcount() // n_grupo ) + 1
#Se saca el salario promedio de cada grupo
afi_sel['SAL_PROM_GRUPO'] = afi_sel.groupby(['CEDULA_COD', 'GRUPO'])['SALARIO'].transform('mean')


In [ ]:
afi_sel[afi_sel['CEDULA_COD']==126].head(60)

In [ ]:
afi_sel.groupby(['CEDULA_COD', 'GRUPO'])['SAL_PROM_GRUPO'].unique()
#     df1 = df1.reset_index()
#     df1['SAL_PROM_GRUPO'] = df1['SAL_PROM_GRUPO'].apply(lambda x: x[0])
#     df1 = df1.sort_values(by = ['CEDULA_COD','SAL_PROM_GRUPO'], ascending=[True,False])

In [ ]:
afi_sel.groupby('CEDULA_COD')['FECHA'].min()
#grupo['ANIO_MIN'] = grupo['CEDULA_COD'].map( grupo.groupby('CEDULA_COD')['ANIO'].min() )
#    grupo['MES_MIN']  = grupo['CEDULA_COD'].map(grupo[ grupo['ANIO']==grupo['ANIO_MIN']].groupby('CEDULA_COD')['MES'].min())